In [ ]:
import json

import numpy as np

from covidcaremap.constants import *
from covidcaremap.data import published_data_path, processed_data_path

## Compute Quantile Breaks

**Note:** This is a work in progress. Currently it just computes the min and max values.

In [ ]:
with open(published_data_path('us_healthcare_capacity-county-CovidCareMap.geojson')) as f:
    by_county = json.loads(f.read())
    
with open(published_data_path('us_healthcare_capacity-state-CovidCareMap.geojson')) as f:
    by_state = json.loads(f.read())
    
with open(published_data_path('us_healthcare_capacity-hrr-CovidCareMap.geojson')) as f:
    by_hrr = json.loads(f.read())
    
with open(published_data_path('us_healthcare_capacity-facility-CovidCareMap.geojson')) as f:
    by_facility = json.loads(f.read())

In [ ]:
def compute_min_maxes(features, columns):
    min_maxes = {}
    for feat in features:
        for prop in columns:
            v = feat['properties'][prop]
            if v is not None and not np.isnan(v):
                # If this is an occupancy value, then it should not 
                # be greater than 1.
                if 'Occupancy' in prop and v > 1.0:
                    v = 1.0
                    
                if prop not in min_maxes:
                    min_maxes[prop] = (v, v)
                else:
                    (curr_min, curr_max) = min_maxes[prop]
                    min_maxes[prop] = (min(curr_min, v), max(curr_max, v)) 
                
    return min_maxes
            

In [ ]:
facility_min_max = compute_min_maxes(by_facility['features'], columns=(
    CCM_FACILITY_COUNT_COLUMNS + 
    list(CCM_FACILITY_OCCUPANCY_COLUMNS.keys())
))

with open(processed_data_path('ccm_facility_min_max_values.json'), 'w') as f:
          f.write(json.dumps(facility_min_max, indent=4))

In [ ]:
county_min_max = compute_min_maxes(by_county['features'], columns=(
    CCM_FACILITY_COUNT_COLUMNS + 
    list(CCM_FACILITY_OCCUPANCY_COLUMNS.keys()) +
    CCM_PER_CAPITA_COLUMNS
))

with open(processed_data_path('ccm_county_min_max_values.json'), 'w') as f:
          f.write(json.dumps(county_min_max, indent=4))

In [ ]:
state_min_max = compute_min_maxes(by_state['features'], columns=(
    CCM_FACILITY_COUNT_COLUMNS + 
    list(CCM_FACILITY_OCCUPANCY_COLUMNS.keys()) +
    CCM_PER_CAPITA_COLUMNS
))

with open(processed_data_path('ccm_state_min_max_values.json'), 'w') as f:
          f.write(json.dumps(state_min_max, indent=4))

In [ ]:
hrr_min_max = compute_min_maxes(by_hrr['features'], columns=(
    CCM_FACILITY_COUNT_COLUMNS + 
    list(CCM_FACILITY_OCCUPANCY_COLUMNS.keys()) +
    CCM_PER_CAPITA_COLUMNS
))

with open(processed_data_path('ccm_hrr_min_max_values.json'), 'w') as f:
          f.write(json.dumps(hrr_min_max, indent=4))